In [ ]:
import os
import glob
import time
import torch
import random
import argparse
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset

import torchvision.models as models
from torchvision import transforms, utils


from PIL import Image
from skimage.transform import resize

import warnings
warnings.filterwarnings("ignore")

### Load Dataset

In [ ]:
class ToTensor(object):
    def __call__(self, sample):
        return torch.tensor(sample, dtype=torch.float32)

class dataset(Dataset):
    def __init__(self, root_dir, dataset_type, img_size, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.file_names = [f for f in glob.glob(os.path.join(root_dir, "*", "*.npz")) \
                            if dataset_type in f]
        self.img_size = img_size
        # self.embeddings = np.load('./embedding.npy')

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        # data_path = os.path.join(self.root_dir, self.file_names[idx])
        data_path = self.file_names[idx]
        data = np.load(data_path)
        image = data["image"].reshape(16, 160, 160)
        resize_image = []
        for idx in range(0, 16):
            #resize_image.append(misc.imresize(image[idx,:,:], (self.img_size, self.img_size)))
             resize_image.append(resize(image[idx,:,:], (self.img_size, self.img_size)))
        resize_image = np.stack(resize_image)
        # image = resize(image, (16, 128, 128))
        # meta_matrix = data["mata_matrix"]
        target = data["target"]
        # structure = data["structure"]
        meta_target = data["meta_target"]

        if meta_target.dtype == np.int8:
            meta_target = meta_target.astype(np.uint8)
        # if meta_structure.dtype == np.int8:
        #     meta_structure = meta_structure.astype(np.uint8)
    
        del data
        if self.transform:
            resize_image = self.transform(resize_image)
            # meta_matrix = self.transform(meta_matrix)
            target = torch.tensor(target, dtype=torch.long)
            meta_target = self.transform(meta_target)
            # meta_structure = self.transform(meta_structure)
            # meta_target = torch.tensor(meta_target, dtype=torch.long)
        return resize_image, target, meta_target

In [ ]:
class ToTensor(object):
    def __call__(self, sample):
        return torch.tensor(sample, dtype=torch.float32)

class dataset(Dataset):
    def __init__(self, root_dir, dataset_type, img_size, transform=None, shuffle=False):
        self.root_dir = root_dir
        self.transform = transform
        self.file_names = [f for f in glob.glob(os.path.join(root_dir, "*", "*.npz")) \
                            if dataset_type in f]
        self.img_size = img_size
        self.shuffle = shuffle
        self.switch = [3,4,5,0,1,2,6,7] 

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        data_path = self.file_names[idx]
        data = np.load(data_path)
        image = data["image"].reshape(16, 160, 160)
        target = data["target"]
        structure = data["structure"]
        meta_target = data["meta_target"]
        meta_structure = data["meta_structure"]

        if self.shuffle:
            context = image[:8, :, :]
            choices = image[8:, :, :]
            indices = list(range(8))
            np.random.shuffle(indices)
            new_target = indices.index(target)
            new_choices = choices[indices, :, :]
            switch_2_rows = np.random.rand()            
            if switch_2_rows < 0.5:                
                context = context[self.switch, :, :]
            image = np.concatenate((context, new_choices))
            target = new_target
        
        resize_image = []
        for idx in range(0, 16):
            resize_image.append(resize(image[idx,:,:], (self.img_size, self.img_size)))
        resize_image = np.stack(resize_image)

        embedding = torch.zeros((6, 300), dtype=torch.float)
        indicator = torch.zeros(1, dtype=torch.float)
        element_idx = 0
    
        del data
        if self.transform:
            resize_image = self.transform(resize_image)
            # meta_matrix = self.transform(meta_matrix)
            target = torch.tensor(target, dtype=torch.long)
            meta_target = self.transform(meta_target)
            meta_structure = self.transform(meta_structure)
            meta_target = torch.tensor(meta_target, dtype=torch.long)
        return resize_image, target, meta_target#, meta_structure, embedding, indicator

#### Experiment Hyperparameters

In [ ]:
class Args:
    
    def __init__(self,):
        self.model = 'WReN'
        self.epochs = 50
        self.batch_size = 16
        self.seed = 12345
        self.device = 2
        self.load_workers = 16
        self.resume = False
        self.path = '/common/home/ls1274/CS536/dat'
        self.save = './ckpt_res/'
        self.img_size = 80
        self.lr = 1e-4
        self.beta1 = 0.9
        self.beta2 = 0.999
        self.epsilon = 1e-8
        #self.meta_alpha = 0.
        self.meta_beta = 10.
        self.perc_train = 20
        self.tag = 1
        
args = Args()

args.cuda = torch.cuda.is_available()
torch.cuda.set_device(args.device)
torch.cuda.manual_seed(args.seed)

if not os.path.exists(args.save):
    os.makedirs(args.save)

#### Load Data

In [ ]:
def gen():
    file_list = []
    for root, dir, files in os.walk("/common/home/kp956/CS536/I-RAVEN/test/"):
        for file in files:
            if file.endswith(".npz"):
                file_list.append(os.path.join(root,file))
    return file_list

class ToTensor(object):
    def __call__(self, sample):
        return torch.tensor(sample, dtype=torch.float32)

train = dataset(args.path, "train", args.img_size, transform=transforms.Compose([ToTensor()]),shuffle=True)
valid = dataset(args.path, "val", args.img_size, transform=transforms.Compose([ToTensor()]))
test = dataset(args.path, "test", args.img_size, transform=transforms.Compose([ToTensor()]))

subset_indices = np.random.choice(len(train), len(train)*args.perc_train // 100, replace=False)
train_subset = Subset(train, subset_indices)

print("Number of samples in original train set =", len(train))
print("Number of samples in train subset =", len(train_subset))
print("All samples are unique =", len(subset_indices) == len(set(subset_indices)))

trainloader = DataLoader(train_subset, batch_size=args.batch_size, shuffle=True, num_workers=16)
validloader = DataLoader(valid, batch_size=args.batch_size, shuffle=False, num_workers=16)
testloader = DataLoader(test, batch_size=args.batch_size, shuffle=False, num_workers=16)

Number of samples in original train set = 24000
Number of samples in train subset = 4800
All samples are unique = True


### Model

In [ ]:
class BasicModel(nn.Module):
    def __init__(self, args):
        super(BasicModel, self).__init__()
        self.name = args.model

    def load_model(self, path, epoch):
        state_dict = torch.load(path+'{}_epoch_{}.pth'.format(self.name, epoch))['state_dict']
        self.load_state_dict(state_dict)

    def save_model(self, path, epoch, acc, loss):
        torch.save({'state_dict': self.state_dict(), 'acc': acc, 'loss': loss}, path+'{}_epoch_{}.pth'.format(self.name, epoch))

    def compute_loss(self, output, target, meta_target):
        pass

    def train_(self, input, target, meta_target):
        self.optimizer.zero_grad()
        output = self(input)
        loss = self.compute_loss(output, target, meta_target)
        loss.backward()
        self.optimizer.step()
        pred = output[0].data.max(1)[1]
        correct = pred.eq(target.data).cpu().sum().numpy()
        accuracy = correct * 100. / target.size()[0]
        return loss.item(), accuracy

    def validate_(self, input, target, meta_target):
        output = self(input)
        loss = self.compute_loss(output, target, meta_target)
        pred = output[0].data.max(1)[1]
        correct = pred.eq(target.data).cpu().sum().numpy()
        accuracy = correct * 100. / target.size()[0]
        return loss.item(), accuracy

    def test_(self, input, target, meta_target):
        output = self(input)
        pred = output[0].data.max(1)[1]
        correct = pred.eq(target.data).cpu().sum().numpy()
        accuracy = correct * 100. / target.size()[0]
        return accuracy

#### WReN

In [ ]:
class conv_module(nn.Module):
    def __init__(self):
        super(conv_module, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=2)
        self.batch_norm1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=2)
        self.batch_norm2 = nn.BatchNorm2d(32)
        self.relu2 = nn.ReLU()
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, stride=2)
        self.batch_norm3 = nn.BatchNorm2d(32)
        self.relu3 = nn.ReLU()
        self.conv4 = nn.Conv2d(32, 32, kernel_size=3, stride=2)
        self.batch_norm4 = nn.BatchNorm2d(32)
        self.relu4 = nn.ReLU()
        # self.fc = nn.Linear(32*4*4, 256)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(self.batch_norm1(x))
        x = self.conv2(x)
        x = self.relu2(self.batch_norm2(x))
        x = self.conv3(x)
        x = self.relu3(self.batch_norm3(x))
        x = self.conv4(x)
        x = self.relu4(self.batch_norm4(x))
        return x.view(-1, 16, 32*4*4)

class relation_module(nn.Module):
    def __init__(self):
        super(relation_module, self).__init__()
        self.fc1 = nn.Linear(256*2, 512)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(512, 512)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(512, 512)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(512, 256)
        self.relu4 = nn.ReLU()

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.relu3(self.fc3(x))
        x = self.relu4(self.fc4(x))
        return x

class mlp_module(nn.Module):
    def __init__(self):
        super(mlp_module, self).__init__()
        self.fc1 = nn.Linear(256, 256)
        self.relu1 = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, 256)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(256, 13)


    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x.view(-1, 8, 13)

class panels_to_embeddings(nn.Module):
    def __init__(self, tag):
        super(panels_to_embeddings, self).__init__()
        self.in_dim = 512
        if tag:
            self.in_dim += 9
        self.fc = nn.Linear(self.in_dim, 256)

    def forward(self, x):
        return self.fc(x.view(-1, self.in_dim))

class WReN(BasicModel):
    def __init__(self, args):
        super(WReN, self).__init__(args)
        self.conv = conv_module()
        self.rn = relation_module()
        self.mlp = mlp_module()
        self.proj = panels_to_embeddings(args.tag)
        self.optimizer = optim.Adam(self.parameters(), lr=args.lr, betas=(args.beta1, args.beta2), eps=args.epsilon)
        self.meta_beta = args.meta_beta 
        self.use_tag = args.tag
        self.use_cuda = args.cuda
        self.tags = self.tag_panels(args.batch_size)

    def tag_panels(self, batch_size):
        tags = []
        for idx in range(0, 16):
            tag = np.zeros([1, 9], dtype=float)
            if idx < 8:
                tag[:, idx] = 1.0
            else:
                tag[:, 8] = 1.0
            tag = torch.tensor(tag, dtype=torch.float).expand(batch_size, -1).unsqueeze(1)
            if self.use_cuda:
                tag = tag.cuda()
            tags.append(tag)
        tags = torch.cat(tags, dim=1)
        return tags

    def group_panel_embeddings(self, embeddings):
        embeddings = embeddings.view(-1, 16, 256)
        embeddings_seq = torch.chunk(embeddings, 16, dim=1)
        context_pairs = []
        for context_idx1 in range(0, 8):
            for context_idx2 in range(0, 8):
                if not context_idx1 == context_idx2:
                    context_pairs.append(torch.cat((embeddings_seq[context_idx1], embeddings_seq[context_idx2]), dim=2))
        context_pairs = torch.cat(context_pairs, dim=1)
        panel_embeddings_pairs = []
        for answer_idx in range(8, len(embeddings_seq)):
            embeddings_pairs = context_pairs
            for context_idx in range(0, 8):
                # In order
                order = torch.cat((embeddings_seq[answer_idx], embeddings_seq[context_idx]), dim=2)
                reverse = torch.cat((embeddings_seq[context_idx], embeddings_seq[answer_idx]), dim=2)
                choice_pairs = torch.cat((order, reverse), dim=1)
                embeddings_pairs = torch.cat((embeddings_pairs, choice_pairs), dim=1)
            panel_embeddings_pairs.append(embeddings_pairs.unsqueeze(1))
        panel_embeddings_pairs = torch.cat(panel_embeddings_pairs, dim=1)
        return panel_embeddings_pairs.view(-1, 8, 72, 512)

    def group_panel_embeddings_batch(self, embeddings):
        embeddings = embeddings.view(-1, 16, 256)
        context_embeddings = embeddings[:,:8,:]
        choice_embeddings = embeddings[:,8:,:]
        context_embeddings_pairs = torch.cat((context_embeddings.unsqueeze(1).expand(-1, 8, -1, -1), context_embeddings.unsqueeze(2).expand(-1, -1, 8, -1)), dim=3).view(-1, 64, 512)
        
        context_embeddings = context_embeddings.unsqueeze(1).expand(-1, 8, -1, -1)
        choice_embeddings = choice_embeddings.unsqueeze(2).expand(-1, -1, 8, -1)
        choice_context_order = torch.cat((context_embeddings, choice_embeddings), dim=3)
        choice_context_reverse = torch.cat((choice_embeddings, context_embeddings), dim=3)
        embedding_paris = [context_embeddings_pairs.unsqueeze(1).expand(-1, 8, -1, -1), choice_context_order, choice_context_reverse]
        return torch.cat(embedding_paris, dim=2).view(-1, 8, 80, 512)


    def rn_sum_features(self, features):
        features = features.view(-1, 8, 80, 256)
        sum_features = torch.sum(features, dim=2)
        return sum_features

    def compute_loss(self, output, target, meta_target):
        pred, meta_pred = output[0], output[1]
        target_loss = F.cross_entropy(pred, target)
        meta_pred = torch.chunk(meta_pred, chunks=12, dim=1)
        meta_target = torch.chunk(meta_target, chunks=12, dim=1)
        meta_target_loss = 0.
        #for idx in range(0, 12):
        #    meta_target_loss += F.binary_cross_entropy(F.sigmoid(meta_pred[idx]), meta_target[idx])
        loss = target_loss #+ self.meta_beta*meta_target_loss / 12.
        return loss

    def forward(self, x):
        panel_features = self.conv(x.view(-1, 1, 80, 80))
        # print(panel_embeddings.size())
        if self.use_tag:
            panel_features = torch.cat((panel_features, self.tags), dim=2)
        panel_embeddings = self.proj(panel_features)
        # panel_embeddings_pairs = self.group_panel_embeddings(panel_embeddings)
        # self.group_panel_embeddings(panel_embeddings)
        panel_embeddings_pairs = self.group_panel_embeddings_batch(panel_embeddings)
        # print(panel_embeddings_pairs.size())
        panel_embedding_features = self.rn(panel_embeddings_pairs.view(-1, 512))
        # print(panel_embedding_features.size())
        sum_features = self.rn_sum_features(panel_embedding_features)
        output = self.mlp(sum_features.view(-1, 256))
        pred = output[:,:,12]
        meta_pred = torch.sum(output[:,:,0:12], dim=1)
        return pred, meta_pred

### Training and Evalution

In [ ]:
def train(epoch, save_file):
    model.train()
    train_loss = 0
    accuracy = 0

    loss_all = 0.0
    acc_all = 0.0
    counter = 0
    for batch_idx, (image, target, meta_target) in enumerate(trainloader):
        counter += 1
        if args.cuda:
            image = image.cuda()
            target = target.cuda()
            meta_target = meta_target.cuda()
            #meta_structure = meta_structure.cuda()
            #embedding = embedding.cuda()
            #indicator = indicator.cuda()
        loss, acc = model.train_(image, target, meta_target)#, meta_structure, embedding, indicator)
        save_str = 'Train: Epoch:{}, Batch:{}, Loss:{:.6f}, Acc:{:.4f}'.format(epoch, batch_idx, loss, acc)
        if counter % 20 == 0:
            print(save_str)
        with open(save_file, 'a') as f:
            f.write(save_str + "\n")
        loss_all += loss
        acc_all += acc
    if counter > 0:
        save_str = "Train_: Avg Training Loss: {:.6f}, Avg Training Acc: {:.6f}".format(
            loss_all/float(counter),
            (acc_all/float(counter))
        )
        print(save_str)
        with open(save_file, 'a') as f:
            f.write(save_str + "\n")
    return loss_all/float(counter), acc_all/float(counter)

def validate(epoch, save_file):
    model.eval()
    val_loss = 0
    accuracy = 0

    loss_all = 0.0
    acc_all = 0.0
    counter = 0
    batch_idx = 0
    for batch_idx, (image, target, meta_target) in enumerate(validloader):
        #print(image)
        counter += 1
        if args.cuda:
            image = image.cuda()
            target = target.cuda()
            meta_target = meta_target.cuda()
            #meta_structure = meta_structure.cuda()
            #embedding = embedding.cuda()
            #indicator = indicator.cuda()
        loss, acc = model.validate_(image, target, meta_target)#, meta_structure, embedding, indicator)
        #print('Validate: Epoch:{}, Batch:{}, Loss:{:.6f}, Acc:{:.4f}.'.format(epoch, batch_idx, loss, acc)) 
        loss_all += loss
        acc_all += acc
    if counter > 0:
        #save_str = "Val_: Total Validation Loss: {:.6f}, Acc: {:.4f}".format((loss_all/float(counter)), (acc_all/float(counter)))
        save_str = "Val_: Total Validation Loss: {:.6f}, Acc: {:.4f}".format((loss_all/float(counter)), (acc_all/float(counter)))
        print(save_str)
        with open(save_file, 'a') as f:
            f.write(save_str + "\n")
    return loss_all/float(counter), acc_all/float(counter)

def test(epoch, save_file):
    model.eval()
    accuracy = 0

    acc_all = 0.0
    counter = 0
    for batch_idx, (image, target, meta_target) in enumerate(testloader):
        counter += 1
        if args.cuda:
            image = image.cuda()
            target = target.cuda()
            meta_target = meta_target.cuda()
            #meta_structure = meta_structure.cuda()
            #embedding = embedding.cuda()
            #indicator = indicator.cuda()
        acc = model.test_(image, target, meta_target)#, meta_structure, embedding, indicator)
        # print('Test: Epoch:{}, Batch:{}, Acc:{:.4f}.'.format(epoch, batch_idx, acc))  
        acc_all += acc
    if counter > 0:
        #save_str = "Test_: Total Testing Acc: {:.4f}".format((acc_all / float(counter)))
        save_str = "Test_: Total Testing Acc: {:.4f}".format((acc_all / float(counter)))
        print(save_str)
        with open(save_file, 'a') as f:
            f.write(save_str + "\n")
    return acc_all/float(counter)


In [ ]:
model = WReN(args)
model = model.cuda()

In [ ]:
SAVE_FILE = "attm1_wren_eps100_dat100_" + time.strftime("%Y-%m-%d_%H:%M:%S", time.gmtime()) + "_" + str(args.perc_train)

for epoch in range(0, args.epochs):
    t0 = time.time()
    train(epoch, SAVE_FILE)
    avg_loss, avg_acc = validate(epoch, SAVE_FILE)
    test_acc = test(epoch, SAVE_FILE)
    model.save_model(args.save, epoch, avg_acc, avg_loss)
    print("Time taken = {:.4f} s\n".format(time.time() - t0))

Train: Epoch:0, Batch:19, Loss:2.031894, Acc:25.0000
Train: Epoch:0, Batch:39, Loss:2.074510, Acc:6.2500
Train: Epoch:0, Batch:59, Loss:2.079844, Acc:12.5000
Train: Epoch:0, Batch:79, Loss:2.095882, Acc:0.0000
Train: Epoch:0, Batch:99, Loss:2.081760, Acc:12.5000
Train: Epoch:0, Batch:119, Loss:2.098263, Acc:25.0000
Train: Epoch:0, Batch:139, Loss:2.094111, Acc:12.5000
Train: Epoch:0, Batch:159, Loss:2.092483, Acc:6.2500
Train: Epoch:0, Batch:179, Loss:2.060433, Acc:12.5000
Train: Epoch:0, Batch:199, Loss:2.126549, Acc:0.0000
Train: Epoch:0, Batch:219, Loss:2.086652, Acc:6.2500
Train: Epoch:0, Batch:239, Loss:2.064678, Acc:12.5000
Train: Epoch:0, Batch:259, Loss:2.106373, Acc:6.2500
Train: Epoch:0, Batch:279, Loss:2.064806, Acc:25.0000
Train: Epoch:0, Batch:299, Loss:2.083160, Acc:0.0000
Train: Epoch:0, Batch:319, Loss:2.095046, Acc:18.7500
Train: Epoch:0, Batch:339, Loss:2.089873, Acc:6.2500
Train: Epoch:0, Batch:359, Loss:2.099159, Acc:12.5000
Train: Epoch:0, Batch:379, Loss:2.072717,

Train: Epoch:1, Batch:1499, Loss:2.073932, Acc:6.2500
Train_: Avg Training Loss: 2.079948, Avg Training Acc: 12.520833
Val_: Total Validation Loss: 2.079293, Acc: 13.6625
Test_: Total Testing Acc: 13.8375
Time taken = 84.7303 s

Train: Epoch:2, Batch:19, Loss:2.084082, Acc:6.2500
Train: Epoch:2, Batch:39, Loss:2.074663, Acc:18.7500
Train: Epoch:2, Batch:59, Loss:2.077444, Acc:12.5000
Train: Epoch:2, Batch:79, Loss:2.083700, Acc:18.7500
Train: Epoch:2, Batch:99, Loss:2.066486, Acc:12.5000
Train: Epoch:2, Batch:119, Loss:2.072962, Acc:18.7500
Train: Epoch:2, Batch:139, Loss:2.088176, Acc:6.2500
Train: Epoch:2, Batch:159, Loss:2.080071, Acc:12.5000
Train: Epoch:2, Batch:179, Loss:2.075761, Acc:25.0000
Train: Epoch:2, Batch:199, Loss:2.082016, Acc:18.7500
Train: Epoch:2, Batch:219, Loss:2.075996, Acc:6.2500
Train: Epoch:2, Batch:239, Loss:2.077737, Acc:6.2500
Train: Epoch:2, Batch:259, Loss:2.069777, Acc:12.5000
Train: Epoch:2, Batch:279, Loss:2.084150, Acc:18.7500
Train: Epoch:2, Batch:29

Train: Epoch:3, Batch:1419, Loss:2.093010, Acc:12.5000
Train: Epoch:3, Batch:1439, Loss:2.076766, Acc:12.5000
Train: Epoch:3, Batch:1459, Loss:2.070419, Acc:18.7500
Train: Epoch:3, Batch:1479, Loss:2.004714, Acc:25.0000
Train: Epoch:3, Batch:1499, Loss:2.091338, Acc:6.2500
Train_: Avg Training Loss: 2.070497, Avg Training Acc: 14.091667
Val_: Total Validation Loss: 2.061622, Acc: 14.5000
Test_: Total Testing Acc: 14.8625
Time taken = 86.5410 s

Train: Epoch:4, Batch:19, Loss:2.101120, Acc:12.5000
Train: Epoch:4, Batch:39, Loss:2.068493, Acc:25.0000
Train: Epoch:4, Batch:59, Loss:2.054405, Acc:12.5000
Train: Epoch:4, Batch:79, Loss:2.150451, Acc:0.0000
Train: Epoch:4, Batch:99, Loss:2.115267, Acc:12.5000
Train: Epoch:4, Batch:119, Loss:2.048069, Acc:31.2500
Train: Epoch:4, Batch:139, Loss:1.965820, Acc:25.0000
Train: Epoch:4, Batch:159, Loss:2.082043, Acc:18.7500
Train: Epoch:4, Batch:179, Loss:2.048445, Acc:12.5000
Train: Epoch:4, Batch:199, Loss:2.089135, Acc:12.5000
Train: Epoch:4, B

Train: Epoch:5, Batch:1339, Loss:2.057037, Acc:12.5000
Train: Epoch:5, Batch:1359, Loss:2.077436, Acc:6.2500
Train: Epoch:5, Batch:1379, Loss:1.933188, Acc:12.5000
Train: Epoch:5, Batch:1399, Loss:2.075646, Acc:6.2500
Train: Epoch:5, Batch:1419, Loss:2.054223, Acc:18.7500
Train: Epoch:5, Batch:1439, Loss:1.948786, Acc:25.0000
Train: Epoch:5, Batch:1459, Loss:2.084891, Acc:0.0000
Train: Epoch:5, Batch:1479, Loss:2.098928, Acc:18.7500
Train: Epoch:5, Batch:1499, Loss:2.026069, Acc:6.2500
Train_: Avg Training Loss: 2.040488, Avg Training Acc: 14.720833
Val_: Total Validation Loss: 2.037249, Acc: 15.0500
Test_: Total Testing Acc: 16.4375
Time taken = 86.6259 s

Train: Epoch:6, Batch:19, Loss:2.054798, Acc:25.0000
Train: Epoch:6, Batch:39, Loss:2.006906, Acc:18.7500
Train: Epoch:6, Batch:59, Loss:2.055825, Acc:6.2500
Train: Epoch:6, Batch:79, Loss:2.018869, Acc:25.0000
Train: Epoch:6, Batch:99, Loss:1.893238, Acc:31.2500
Train: Epoch:6, Batch:119, Loss:2.049445, Acc:6.2500
Train: Epoch:6, B

Train: Epoch:7, Batch:1259, Loss:1.999537, Acc:12.5000
Train: Epoch:7, Batch:1279, Loss:2.091083, Acc:0.0000
Train: Epoch:7, Batch:1299, Loss:2.035747, Acc:31.2500
Train: Epoch:7, Batch:1319, Loss:2.092860, Acc:12.5000
Train: Epoch:7, Batch:1339, Loss:1.948672, Acc:25.0000
Train: Epoch:7, Batch:1359, Loss:2.005517, Acc:12.5000
Train: Epoch:7, Batch:1379, Loss:1.984259, Acc:31.2500
Train: Epoch:7, Batch:1399, Loss:1.865986, Acc:43.7500
Train: Epoch:7, Batch:1419, Loss:1.988010, Acc:12.5000
Train: Epoch:7, Batch:1439, Loss:2.083559, Acc:0.0000
Train: Epoch:7, Batch:1459, Loss:1.949888, Acc:31.2500
Train: Epoch:7, Batch:1479, Loss:1.984375, Acc:25.0000
Train: Epoch:7, Batch:1499, Loss:1.936361, Acc:25.0000
Train_: Avg Training Loss: 2.014654, Avg Training Acc: 16.866667
Val_: Total Validation Loss: 2.018353, Acc: 16.6000
Test_: Total Testing Acc: 17.3875
Time taken = 82.5824 s

Train: Epoch:8, Batch:19, Loss:2.070112, Acc:12.5000
Train: Epoch:8, Batch:39, Loss:2.098882, Acc:12.5000
Train:

...

Train: Epoch:91, Batch:299, Loss:0.054135, Acc:100.0000
Train: Epoch:91, Batch:319, Loss:0.265963, Acc:87.5000
Train: Epoch:91, Batch:339, Loss:0.073874, Acc:100.0000
Train: Epoch:91, Batch:359, Loss:0.011036, Acc:100.0000
Train: Epoch:91, Batch:379, Loss:0.001122, Acc:100.0000
Train: Epoch:91, Batch:399, Loss:0.316639, Acc:93.7500
Train: Epoch:91, Batch:419, Loss:0.003757, Acc:100.0000
Train: Epoch:91, Batch:439, Loss:0.023313, Acc:100.0000
Train: Epoch:91, Batch:459, Loss:0.047768, Acc:100.0000
Train: Epoch:91, Batch:479, Loss:0.023460, Acc:100.0000
Train: Epoch:91, Batch:499, Loss:0.092158, Acc:100.0000
Train: Epoch:91, Batch:519, Loss:0.003165, Acc:100.0000
Train: Epoch:91, Batch:539, Loss:0.021398, Acc:100.0000
Train: Epoch:91, Batch:559, Loss:0.219303, Acc:87.5000
Train: Epoch:91, Batch:579, Loss:0.090874, Acc:93.7500
Train: Epoch:91, Batch:599, Loss:0.085839, Acc:93.7500
Train: Epoch:91, Batch:619, Loss:0.247563, Acc:93.7500
Train: Epoch:91, Batch:639, Loss:0.199550, Acc:93.7500

Train: Epoch:93, Batch:119, Loss:0.299032, Acc:81.2500
Train: Epoch:93, Batch:139, Loss:0.122816, Acc:93.7500
Train: Epoch:93, Batch:159, Loss:0.021033, Acc:100.0000
Train: Epoch:93, Batch:179, Loss:0.123517, Acc:87.5000
Train: Epoch:93, Batch:199, Loss:0.037480, Acc:100.0000
Train: Epoch:93, Batch:219, Loss:0.031460, Acc:100.0000
Train: Epoch:93, Batch:239, Loss:0.032118, Acc:100.0000
Train: Epoch:93, Batch:259, Loss:0.021572, Acc:100.0000
Train: Epoch:93, Batch:279, Loss:0.059825, Acc:100.0000
Train: Epoch:93, Batch:299, Loss:0.035688, Acc:100.0000
Train: Epoch:93, Batch:319, Loss:0.016106, Acc:100.0000
Train: Epoch:93, Batch:339, Loss:0.013789, Acc:100.0000
Train: Epoch:93, Batch:359, Loss:0.519868, Acc:81.2500
Train: Epoch:93, Batch:379, Loss:0.065354, Acc:100.0000
Train: Epoch:93, Batch:399, Loss:0.051573, Acc:100.0000
Train: Epoch:93, Batch:419, Loss:0.322716, Acc:93.7500
Train: Epoch:93, Batch:439, Loss:0.259144, Acc:81.2500
Train: Epoch:93, Batch:459, Loss:0.007776, Acc:100.000

Train: Epoch:94, Batch:1499, Loss:0.066791, Acc:100.0000
Train_: Avg Training Loss: 0.125174, Avg Training Acc: 95.991667
Val_: Total Validation Loss: 6.970471, Acc: 20.1375
Test_: Total Testing Acc: 20.7500
Time taken = 91.2255 s

Train: Epoch:95, Batch:19, Loss:0.035882, Acc:100.0000
Train: Epoch:95, Batch:39, Loss:0.038084, Acc:100.0000
Train: Epoch:95, Batch:59, Loss:0.011067, Acc:100.0000
Train: Epoch:95, Batch:79, Loss:0.093538, Acc:93.7500
Train: Epoch:95, Batch:99, Loss:0.068102, Acc:93.7500
Train: Epoch:95, Batch:119, Loss:0.057332, Acc:100.0000
Train: Epoch:95, Batch:139, Loss:0.138394, Acc:93.7500
Train: Epoch:95, Batch:159, Loss:0.094980, Acc:93.7500
Train: Epoch:95, Batch:179, Loss:0.102601, Acc:93.7500
Train: Epoch:95, Batch:199, Loss:0.064208, Acc:93.7500
Train: Epoch:95, Batch:219, Loss:0.055170, Acc:100.0000
Train: Epoch:95, Batch:239, Loss:0.044999, Acc:100.0000
Train: Epoch:95, Batch:259, Loss:0.370387, Acc:93.7500
Train: Epoch:95, Batch:279, Loss:0.071953, Acc:93.75

Train: Epoch:96, Batch:1319, Loss:0.063466, Acc:100.0000
Train: Epoch:96, Batch:1339, Loss:0.234006, Acc:87.5000
Train: Epoch:96, Batch:1359, Loss:0.425345, Acc:87.5000
Train: Epoch:96, Batch:1379, Loss:0.040097, Acc:100.0000
Train: Epoch:96, Batch:1399, Loss:0.030806, Acc:100.0000
Train: Epoch:96, Batch:1419, Loss:0.021524, Acc:100.0000
Train: Epoch:96, Batch:1439, Loss:0.148292, Acc:93.7500
Train: Epoch:96, Batch:1459, Loss:0.011439, Acc:100.0000
Train: Epoch:96, Batch:1479, Loss:0.158873, Acc:93.7500
Train: Epoch:96, Batch:1499, Loss:0.197719, Acc:93.7500
Train_: Avg Training Loss: 0.124143, Avg Training Acc: 95.962500
Val_: Total Validation Loss: 7.253457, Acc: 20.4625
Test_: Total Testing Acc: 20.6000
Time taken = 87.2930 s

Train: Epoch:97, Batch:19, Loss:0.092604, Acc:93.7500
Train: Epoch:97, Batch:39, Loss:0.064829, Acc:100.0000
Train: Epoch:97, Batch:59, Loss:0.083679, Acc:100.0000
Train: Epoch:97, Batch:79, Loss:0.008740, Acc:100.0000
Train: Epoch:97, Batch:99, Loss:0.175242,

Train: Epoch:98, Batch:1139, Loss:0.169591, Acc:87.5000
Train: Epoch:98, Batch:1159, Loss:0.023928, Acc:100.0000
Train: Epoch:98, Batch:1179, Loss:0.621343, Acc:87.5000
Train: Epoch:98, Batch:1199, Loss:0.022507, Acc:100.0000
Train: Epoch:98, Batch:1219, Loss:0.176665, Acc:87.5000
Train: Epoch:98, Batch:1239, Loss:0.061909, Acc:100.0000
Train: Epoch:98, Batch:1259, Loss:0.038471, Acc:100.0000
Train: Epoch:98, Batch:1279, Loss:0.020773, Acc:100.0000
Train: Epoch:98, Batch:1299, Loss:0.085353, Acc:93.7500
Train: Epoch:98, Batch:1319, Loss:0.237965, Acc:93.7500
Train: Epoch:98, Batch:1339, Loss:0.025314, Acc:100.0000
Train: Epoch:98, Batch:1359, Loss:0.022039, Acc:100.0000
Train: Epoch:98, Batch:1379, Loss:0.057926, Acc:93.7500
Train: Epoch:98, Batch:1399, Loss:0.149857, Acc:87.5000
Train: Epoch:98, Batch:1419, Loss:0.745770, Acc:81.2500
Train: Epoch:98, Batch:1439, Loss:0.076646, Acc:100.0000
Train: Epoch:98, Batch:1459, Loss:0.070168, Acc:100.0000
Train: Epoch:98, Batch:1479, Loss:0.165

In [ ]:
#Re-running the model for 80% of data
SAVE_FILE = "attm1_wren_eps50_dat80_" + time.strftime("%Y-%m-%d_%H:%M:%S", time.gmtime()) + "_" + str(args.perc_train)

for epoch in range(0, args.epochs):
    t0 = time.time()
    train(epoch, SAVE_FILE)
    avg_loss, avg_acc = validate(epoch, SAVE_FILE)
    test_acc = test(epoch, SAVE_FILE)
    model.save_model(args.save, epoch, avg_acc, avg_loss)
    print("Time taken = {:.4f} s\n".format(time.time() - t0))

Train: Epoch:0, Batch:19, Loss:2.080613, Acc:12.5000
Train: Epoch:0, Batch:39, Loss:2.145501, Acc:6.2500
Train: Epoch:0, Batch:59, Loss:2.140167, Acc:0.0000
Train: Epoch:0, Batch:79, Loss:2.044687, Acc:18.7500
Train: Epoch:0, Batch:99, Loss:2.073349, Acc:12.5000
Train: Epoch:0, Batch:119, Loss:2.113395, Acc:6.2500
Train: Epoch:0, Batch:139, Loss:2.112019, Acc:12.5000
Train: Epoch:0, Batch:159, Loss:2.108823, Acc:6.2500
Train: Epoch:0, Batch:179, Loss:2.071123, Acc:37.5000
Train: Epoch:0, Batch:199, Loss:2.113931, Acc:0.0000
Train: Epoch:0, Batch:219, Loss:2.069161, Acc:18.7500
Train: Epoch:0, Batch:239, Loss:2.047858, Acc:18.7500
Train: Epoch:0, Batch:259, Loss:2.039830, Acc:25.0000
Train: Epoch:0, Batch:279, Loss:2.095494, Acc:12.5000
Train: Epoch:0, Batch:299, Loss:2.063690, Acc:31.2500
Train: Epoch:0, Batch:319, Loss:2.101639, Acc:12.5000
Train: Epoch:0, Batch:339, Loss:2.105389, Acc:6.2500
Train: Epoch:0, Batch:359, Loss:2.091220, Acc:0.0000
Train: Epoch:0, Batch:379, Loss:2.065395

Train: Epoch:2, Batch:559, Loss:2.086379, Acc:6.2500
Train: Epoch:2, Batch:579, Loss:2.066357, Acc:18.7500
Train: Epoch:2, Batch:599, Loss:2.084269, Acc:6.2500
Train: Epoch:2, Batch:619, Loss:2.047251, Acc:25.0000
Train: Epoch:2, Batch:639, Loss:2.049121, Acc:12.5000
Train: Epoch:2, Batch:659, Loss:2.123589, Acc:25.0000
Train: Epoch:2, Batch:679, Loss:2.069908, Acc:12.5000
Train: Epoch:2, Batch:699, Loss:2.101876, Acc:18.7500
Train: Epoch:2, Batch:719, Loss:2.080545, Acc:12.5000
Train: Epoch:2, Batch:739, Loss:2.072289, Acc:6.2500
Train: Epoch:2, Batch:759, Loss:2.059987, Acc:31.2500
Train: Epoch:2, Batch:779, Loss:2.066971, Acc:12.5000
Train: Epoch:2, Batch:799, Loss:2.087575, Acc:18.7500
Train: Epoch:2, Batch:819, Loss:2.110792, Acc:6.2500
Train: Epoch:2, Batch:839, Loss:2.064559, Acc:18.7500
Train: Epoch:2, Batch:859, Loss:2.055882, Acc:18.7500
Train: Epoch:2, Batch:879, Loss:2.089026, Acc:6.2500
Train: Epoch:2, Batch:899, Loss:2.098581, Acc:0.0000
Train: Epoch:2, Batch:919, Loss:2.

Train: Epoch:4, Batch:1079, Loss:2.031697, Acc:18.7500
Train: Epoch:4, Batch:1099, Loss:2.020216, Acc:6.2500
Train: Epoch:4, Batch:1119, Loss:2.026558, Acc:12.5000
Train: Epoch:4, Batch:1139, Loss:2.130496, Acc:6.2500
Train: Epoch:4, Batch:1159, Loss:2.032772, Acc:18.7500
Train: Epoch:4, Batch:1179, Loss:2.121124, Acc:0.0000
Train: Epoch:4, Batch:1199, Loss:2.153809, Acc:6.2500
Train_: Avg Training Loss: 2.057095, Avg Training Acc: 14.119792
Val_: Total Validation Loss: 2.047460, Acc: 14.2375
Test_: Total Testing Acc: 13.7500
Time taken = 84.2162 s

Train: Epoch:5, Batch:19, Loss:2.071627, Acc:6.2500
Train: Epoch:5, Batch:39, Loss:2.047249, Acc:6.2500
Train: Epoch:5, Batch:59, Loss:2.002061, Acc:12.5000
Train: Epoch:5, Batch:79, Loss:2.115179, Acc:12.5000
Train: Epoch:5, Batch:99, Loss:2.111392, Acc:12.5000
Train: Epoch:5, Batch:119, Loss:2.150976, Acc:12.5000
Train: Epoch:5, Batch:139, Loss:2.081205, Acc:18.7500
Train: Epoch:5, Batch:159, Loss:2.067927, Acc:12.5000
Train: Epoch:5, Bat

Train: Epoch:7, Batch:339, Loss:1.978392, Acc:31.2500
Train: Epoch:7, Batch:359, Loss:2.174053, Acc:0.0000
Train: Epoch:7, Batch:379, Loss:2.098855, Acc:0.0000
Train: Epoch:7, Batch:399, Loss:2.075266, Acc:0.0000
Train: Epoch:7, Batch:419, Loss:2.072859, Acc:18.7500
Train: Epoch:7, Batch:439, Loss:2.023899, Acc:25.0000
Train: Epoch:7, Batch:459, Loss:2.046278, Acc:18.7500
Train: Epoch:7, Batch:479, Loss:1.987537, Acc:18.7500
Train: Epoch:7, Batch:499, Loss:2.037696, Acc:12.5000
Train: Epoch:7, Batch:519, Loss:2.094856, Acc:6.2500
Train: Epoch:7, Batch:539, Loss:2.031458, Acc:12.5000
Train: Epoch:7, Batch:559, Loss:2.054110, Acc:18.7500
Train: Epoch:7, Batch:579, Loss:1.950288, Acc:18.7500
Train: Epoch:7, Batch:599, Loss:2.048531, Acc:31.2500
Train: Epoch:7, Batch:619, Loss:2.102767, Acc:12.5000
Train: Epoch:7, Batch:639, Loss:2.086661, Acc:18.7500
Train: Epoch:7, Batch:659, Loss:2.073187, Acc:6.2500
Train: Epoch:7, Batch:679, Loss:2.067064, Acc:6.2500
Train: Epoch:7, Batch:699, Loss:1.

Train: Epoch:9, Batch:859, Loss:1.995655, Acc:12.5000
Train: Epoch:9, Batch:879, Loss:1.952897, Acc:25.0000
Train: Epoch:9, Batch:899, Loss:2.051644, Acc:18.7500
Train: Epoch:9, Batch:919, Loss:1.906612, Acc:31.2500
Train: Epoch:9, Batch:939, Loss:1.941232, Acc:18.7500
Train: Epoch:9, Batch:959, Loss:2.098878, Acc:18.7500
Train: Epoch:9, Batch:979, Loss:1.977812, Acc:18.7500
Train: Epoch:9, Batch:999, Loss:2.160996, Acc:0.0000
Train: Epoch:9, Batch:1019, Loss:1.911491, Acc:25.0000
Train: Epoch:9, Batch:1039, Loss:1.957476, Acc:37.5000
Train: Epoch:9, Batch:1059, Loss:2.024504, Acc:18.7500
Train: Epoch:9, Batch:1079, Loss:1.945155, Acc:18.7500
Train: Epoch:9, Batch:1099, Loss:1.856532, Acc:37.5000
Train: Epoch:9, Batch:1119, Loss:2.053175, Acc:6.2500
Train: Epoch:9, Batch:1139, Loss:2.121594, Acc:6.2500
Train: Epoch:9, Batch:1159, Loss:1.953513, Acc:6.2500
Train: Epoch:9, Batch:1179, Loss:1.907772, Acc:18.7500
Train: Epoch:9, Batch:1199, Loss:2.021651, Acc:6.2500
Train_: Avg Training Lo

...

Train: Epoch:40, Batch:519, Loss:0.104131, Acc:100.0000
Train: Epoch:40, Batch:539, Loss:0.578263, Acc:87.5000
Train: Epoch:40, Batch:559, Loss:0.444504, Acc:81.2500
Train: Epoch:40, Batch:579, Loss:0.275090, Acc:81.2500
Train: Epoch:40, Batch:599, Loss:0.227982, Acc:81.2500
Train: Epoch:40, Batch:619, Loss:0.137819, Acc:100.0000
Train: Epoch:40, Batch:639, Loss:0.295252, Acc:87.5000
Train: Epoch:40, Batch:659, Loss:0.301302, Acc:87.5000
Train: Epoch:40, Batch:679, Loss:0.344038, Acc:87.5000
Train: Epoch:40, Batch:699, Loss:0.233484, Acc:100.0000
Train: Epoch:40, Batch:719, Loss:0.394429, Acc:81.2500
Train: Epoch:40, Batch:739, Loss:0.167495, Acc:93.7500
Train: Epoch:40, Batch:759, Loss:0.484994, Acc:75.0000
Train: Epoch:40, Batch:779, Loss:0.283084, Acc:93.7500
Train: Epoch:40, Batch:799, Loss:0.296610, Acc:87.5000
Train: Epoch:40, Batch:819, Loss:0.408115, Acc:87.5000
Train: Epoch:40, Batch:839, Loss:0.327650, Acc:93.7500
Train: Epoch:40, Batch:859, Loss:0.116523, Acc:100.0000
Train:

Train: Epoch:42, Batch:959, Loss:0.463021, Acc:87.5000
Train: Epoch:42, Batch:979, Loss:0.131739, Acc:93.7500
Train: Epoch:42, Batch:999, Loss:0.336668, Acc:81.2500
Train: Epoch:42, Batch:1019, Loss:0.166426, Acc:93.7500
Train: Epoch:42, Batch:1039, Loss:0.289257, Acc:81.2500
Train: Epoch:42, Batch:1059, Loss:0.511733, Acc:87.5000
Train: Epoch:42, Batch:1079, Loss:0.102719, Acc:100.0000
Train: Epoch:42, Batch:1099, Loss:0.536382, Acc:81.2500
Train: Epoch:42, Batch:1119, Loss:0.237615, Acc:93.7500
Train: Epoch:42, Batch:1139, Loss:0.151146, Acc:100.0000
Train: Epoch:42, Batch:1159, Loss:0.474553, Acc:75.0000
Train: Epoch:42, Batch:1179, Loss:0.416529, Acc:75.0000
Train: Epoch:42, Batch:1199, Loss:0.263766, Acc:87.5000
Train_: Avg Training Loss: 0.285202, Avg Training Acc: 89.942708
Val_: Total Validation Loss: 6.172920, Acc: 19.8375
Test_: Total Testing Acc: 19.6375
Time taken = 83.9180 s

Train: Epoch:43, Batch:19, Loss:0.207312, Acc:87.5000
Train: Epoch:43, Batch:39, Loss:0.275699, Ac

Train: Epoch:45, Batch:139, Loss:0.253537, Acc:93.7500
Train: Epoch:45, Batch:159, Loss:0.016184, Acc:100.0000
Train: Epoch:45, Batch:179, Loss:0.008750, Acc:100.0000
Train: Epoch:45, Batch:199, Loss:0.084697, Acc:100.0000
Train: Epoch:45, Batch:219, Loss:0.121454, Acc:93.7500
Train: Epoch:45, Batch:239, Loss:0.023968, Acc:100.0000
Train: Epoch:45, Batch:259, Loss:0.435433, Acc:81.2500
Train: Epoch:45, Batch:279, Loss:0.316261, Acc:87.5000
Train: Epoch:45, Batch:299, Loss:0.062803, Acc:100.0000
Train: Epoch:45, Batch:319, Loss:0.226993, Acc:93.7500
Train: Epoch:45, Batch:339, Loss:0.146273, Acc:93.7500
Train: Epoch:45, Batch:359, Loss:0.190514, Acc:93.7500
Train: Epoch:45, Batch:379, Loss:0.363979, Acc:93.7500
Train: Epoch:45, Batch:399, Loss:0.063092, Acc:100.0000
Train: Epoch:45, Batch:419, Loss:0.140433, Acc:100.0000
Train: Epoch:45, Batch:439, Loss:0.094115, Acc:93.7500
Train: Epoch:45, Batch:459, Loss:0.087349, Acc:100.0000
Train: Epoch:45, Batch:479, Loss:0.142714, Acc:93.7500
Tr

Train: Epoch:47, Batch:579, Loss:0.076271, Acc:100.0000
Train: Epoch:47, Batch:599, Loss:0.150769, Acc:93.7500
Train: Epoch:47, Batch:619, Loss:0.174794, Acc:93.7500
Train: Epoch:47, Batch:639, Loss:0.015972, Acc:100.0000
Train: Epoch:47, Batch:659, Loss:0.004781, Acc:100.0000
Train: Epoch:47, Batch:679, Loss:0.280398, Acc:93.7500
Train: Epoch:47, Batch:699, Loss:0.168551, Acc:100.0000
Train: Epoch:47, Batch:719, Loss:0.636513, Acc:81.2500
Train: Epoch:47, Batch:739, Loss:0.348791, Acc:81.2500
Train: Epoch:47, Batch:759, Loss:0.257911, Acc:93.7500
Train: Epoch:47, Batch:779, Loss:0.585915, Acc:75.0000
Train: Epoch:47, Batch:799, Loss:0.365107, Acc:81.2500
Train: Epoch:47, Batch:819, Loss:0.097366, Acc:100.0000
Train: Epoch:47, Batch:839, Loss:0.057277, Acc:100.0000
Train: Epoch:47, Batch:859, Loss:0.612711, Acc:87.5000
Train: Epoch:47, Batch:879, Loss:0.274621, Acc:87.5000
Train: Epoch:47, Batch:899, Loss:0.322289, Acc:87.5000
Train: Epoch:47, Batch:919, Loss:0.159405, Acc:93.7500
Trai

Train: Epoch:49, Batch:1019, Loss:0.092168, Acc:100.0000
Train: Epoch:49, Batch:1039, Loss:0.090685, Acc:93.7500
Train: Epoch:49, Batch:1059, Loss:0.701138, Acc:75.0000
Train: Epoch:49, Batch:1079, Loss:0.175462, Acc:87.5000
Train: Epoch:49, Batch:1099, Loss:0.487976, Acc:68.7500
Train: Epoch:49, Batch:1119, Loss:0.056994, Acc:100.0000
Train: Epoch:49, Batch:1139, Loss:0.075646, Acc:100.0000
Train: Epoch:49, Batch:1159, Loss:0.181574, Acc:87.5000
Train: Epoch:49, Batch:1179, Loss:0.425812, Acc:75.0000
Train: Epoch:49, Batch:1199, Loss:0.134414, Acc:93.7500
Train_: Avg Training Loss: 0.229392, Avg Training Acc: 92.250000
Val_: Total Validation Loss: 5.812355, Acc: 20.5125
Test_: Total Testing Acc: 19.9375
Time taken = 83.6906 s



In [ ]:
#Re-running the model for 60% of data
SAVE_FILE = "attm1_wren_eps50_dat60_" + time.strftime("%Y-%m-%d_%H:%M:%S", time.gmtime()) + "_" + str(args.perc_train)

for epoch in range(0, args.epochs):
    t0 = time.time()
    train(epoch, SAVE_FILE)
    avg_loss, avg_acc = validate(epoch, SAVE_FILE)
    test_acc = test(epoch, SAVE_FILE)
    model.save_model(args.save, epoch, avg_acc, avg_loss)
    print("Time taken = {:.4f} s\n".format(time.time() - t0))

Train: Epoch:0, Batch:19, Loss:2.103005, Acc:0.0000
Train: Epoch:0, Batch:39, Loss:2.130304, Acc:12.5000
Train: Epoch:0, Batch:59, Loss:2.062837, Acc:12.5000
Train: Epoch:0, Batch:79, Loss:2.086526, Acc:12.5000
Train: Epoch:0, Batch:99, Loss:2.044057, Acc:12.5000
Train: Epoch:0, Batch:119, Loss:2.084743, Acc:18.7500
Train: Epoch:0, Batch:139, Loss:2.047260, Acc:12.5000
Train: Epoch:0, Batch:159, Loss:2.070482, Acc:12.5000
Train: Epoch:0, Batch:179, Loss:2.084278, Acc:6.2500
Train: Epoch:0, Batch:199, Loss:2.091755, Acc:6.2500
Train: Epoch:0, Batch:219, Loss:2.077121, Acc:6.2500
Train: Epoch:0, Batch:239, Loss:2.071730, Acc:6.2500
Train: Epoch:0, Batch:259, Loss:2.068861, Acc:6.2500
Train: Epoch:0, Batch:279, Loss:2.082850, Acc:6.2500
Train: Epoch:0, Batch:299, Loss:2.041378, Acc:37.5000
Train: Epoch:0, Batch:319, Loss:2.080968, Acc:18.7500
Train: Epoch:0, Batch:339, Loss:2.073967, Acc:18.7500
Train: Epoch:0, Batch:359, Loss:2.102285, Acc:6.2500
Train: Epoch:0, Batch:379, Loss:2.122108,

Train: Epoch:3, Batch:199, Loss:2.081917, Acc:6.2500
Train: Epoch:3, Batch:219, Loss:2.093521, Acc:12.5000
Train: Epoch:3, Batch:239, Loss:2.068979, Acc:6.2500
Train: Epoch:3, Batch:259, Loss:2.085259, Acc:12.5000
Train: Epoch:3, Batch:279, Loss:2.083678, Acc:6.2500
Train: Epoch:3, Batch:299, Loss:2.069453, Acc:18.7500
Train: Epoch:3, Batch:319, Loss:2.080578, Acc:12.5000
Train: Epoch:3, Batch:339, Loss:2.094726, Acc:6.2500
Train: Epoch:3, Batch:359, Loss:2.063398, Acc:31.2500
Train: Epoch:3, Batch:379, Loss:2.080241, Acc:12.5000
Train: Epoch:3, Batch:399, Loss:2.090984, Acc:6.2500
Train: Epoch:3, Batch:419, Loss:2.087300, Acc:6.2500
Train: Epoch:3, Batch:439, Loss:2.082950, Acc:12.5000
Train: Epoch:3, Batch:459, Loss:2.079249, Acc:12.5000
Train: Epoch:3, Batch:479, Loss:2.080463, Acc:6.2500
Train: Epoch:3, Batch:499, Loss:2.079788, Acc:12.5000
Train: Epoch:3, Batch:519, Loss:2.088940, Acc:6.2500
Train: Epoch:3, Batch:539, Loss:2.089889, Acc:12.5000
Train: Epoch:3, Batch:559, Loss:2.08

Train: Epoch:6, Batch:379, Loss:2.088343, Acc:6.2500
Train: Epoch:6, Batch:399, Loss:2.071031, Acc:18.7500
Train: Epoch:6, Batch:419, Loss:2.041680, Acc:6.2500
Train: Epoch:6, Batch:439, Loss:2.127739, Acc:6.2500
Train: Epoch:6, Batch:459, Loss:2.066629, Acc:6.2500
Train: Epoch:6, Batch:479, Loss:2.098747, Acc:12.5000
Train: Epoch:6, Batch:499, Loss:2.090677, Acc:12.5000
Train: Epoch:6, Batch:519, Loss:2.070005, Acc:6.2500
Train: Epoch:6, Batch:539, Loss:2.093562, Acc:6.2500
Train: Epoch:6, Batch:559, Loss:2.081766, Acc:12.5000
Train: Epoch:6, Batch:579, Loss:2.072285, Acc:18.7500
Train: Epoch:6, Batch:599, Loss:2.058687, Acc:25.0000
Train: Epoch:6, Batch:619, Loss:2.079662, Acc:31.2500
Train: Epoch:6, Batch:639, Loss:2.075725, Acc:0.0000
Train: Epoch:6, Batch:659, Loss:2.129739, Acc:18.7500
Train: Epoch:6, Batch:679, Loss:2.074755, Acc:0.0000
Train: Epoch:6, Batch:699, Loss:2.078117, Acc:12.5000
Train: Epoch:6, Batch:719, Loss:2.069077, Acc:0.0000
Train: Epoch:6, Batch:739, Loss:2.098

Train: Epoch:9, Batch:559, Loss:2.089320, Acc:18.7500
Train: Epoch:9, Batch:579, Loss:1.995246, Acc:12.5000
Train: Epoch:9, Batch:599, Loss:2.077393, Acc:25.0000
Train: Epoch:9, Batch:619, Loss:2.033194, Acc:18.7500
Train: Epoch:9, Batch:639, Loss:1.954461, Acc:12.5000
Train: Epoch:9, Batch:659, Loss:1.998805, Acc:18.7500
Train: Epoch:9, Batch:679, Loss:1.929533, Acc:12.5000
Train: Epoch:9, Batch:699, Loss:2.053923, Acc:6.2500
Train: Epoch:9, Batch:719, Loss:2.027260, Acc:25.0000
Train: Epoch:9, Batch:739, Loss:2.053996, Acc:12.5000
Train: Epoch:9, Batch:759, Loss:2.003137, Acc:25.0000
Train: Epoch:9, Batch:779, Loss:2.029309, Acc:18.7500
Train: Epoch:9, Batch:799, Loss:2.104760, Acc:6.2500
Train: Epoch:9, Batch:819, Loss:2.044729, Acc:6.2500
Train: Epoch:9, Batch:839, Loss:1.907280, Acc:12.5000
Train: Epoch:9, Batch:859, Loss:2.075466, Acc:12.5000
Train: Epoch:9, Batch:879, Loss:2.062683, Acc:25.0000
Train: Epoch:9, Batch:899, Loss:2.034916, Acc:12.5000
Train_: Avg Training Loss: 2.04

Train: Epoch:12, Batch:679, Loss:1.942242, Acc:25.0000
Train: Epoch:12, Batch:699, Loss:1.908036, Acc:12.5000
Train: Epoch:12, Batch:719, Loss:2.207691, Acc:18.7500
Train: Epoch:12, Batch:739, Loss:2.019511, Acc:12.5000
Train: Epoch:12, Batch:759, Loss:1.934217, Acc:25.0000
Train: Epoch:12, Batch:779, Loss:1.932560, Acc:25.0000
Train: Epoch:12, Batch:799, Loss:2.110701, Acc:6.2500
Train: Epoch:12, Batch:819, Loss:2.066655, Acc:12.5000
Train: Epoch:12, Batch:839, Loss:2.165287, Acc:12.5000
Train: Epoch:12, Batch:859, Loss:1.998028, Acc:25.0000
Train: Epoch:12, Batch:879, Loss:2.015166, Acc:12.5000
Train: Epoch:12, Batch:899, Loss:2.029538, Acc:6.2500
Train_: Avg Training Loss: 2.004028, Avg Training Acc: 17.597222
Val_: Total Validation Loss: 2.031744, Acc: 15.6375
Test_: Total Testing Acc: 15.8375
Time taken = 72.3469 s

Train: Epoch:13, Batch:19, Loss:1.997445, Acc:0.0000
Train: Epoch:13, Batch:39, Loss:1.828321, Acc:43.7500
Train: Epoch:13, Batch:59, Loss:2.038394, Acc:18.7500
Train:

...

Train: Epoch:37, Batch:539, Loss:0.156613, Acc:100.0000
Train: Epoch:37, Batch:559, Loss:0.515080, Acc:81.2500
Train: Epoch:37, Batch:579, Loss:0.162473, Acc:100.0000
Train: Epoch:37, Batch:599, Loss:0.238882, Acc:100.0000
Train: Epoch:37, Batch:619, Loss:0.718796, Acc:87.5000
Train: Epoch:37, Batch:639, Loss:0.345773, Acc:81.2500
Train: Epoch:37, Batch:659, Loss:0.556400, Acc:75.0000
Train: Epoch:37, Batch:679, Loss:0.336888, Acc:87.5000
Train: Epoch:37, Batch:699, Loss:0.575834, Acc:81.2500
Train: Epoch:37, Batch:719, Loss:0.584493, Acc:87.5000
Train: Epoch:37, Batch:739, Loss:0.464883, Acc:81.2500
Train: Epoch:37, Batch:759, Loss:0.543920, Acc:81.2500
Train: Epoch:37, Batch:779, Loss:0.466951, Acc:81.2500
Train: Epoch:37, Batch:799, Loss:0.026961, Acc:100.0000
Train: Epoch:37, Batch:819, Loss:0.315941, Acc:87.5000
Train: Epoch:37, Batch:839, Loss:0.451535, Acc:87.5000
Train: Epoch:37, Batch:859, Loss:0.185948, Acc:93.7500
Train: Epoch:37, Batch:879, Loss:0.362818, Acc:87.5000
Train:

Train: Epoch:40, Batch:639, Loss:0.292315, Acc:93.7500
Train: Epoch:40, Batch:659, Loss:0.859691, Acc:81.2500
Train: Epoch:40, Batch:679, Loss:0.128976, Acc:100.0000
Train: Epoch:40, Batch:699, Loss:0.248383, Acc:87.5000
Train: Epoch:40, Batch:719, Loss:0.216239, Acc:93.7500
Train: Epoch:40, Batch:739, Loss:0.309813, Acc:87.5000
Train: Epoch:40, Batch:759, Loss:0.478085, Acc:75.0000
Train: Epoch:40, Batch:779, Loss:0.346034, Acc:81.2500
Train: Epoch:40, Batch:799, Loss:0.484530, Acc:75.0000
Train: Epoch:40, Batch:819, Loss:0.204053, Acc:87.5000
Train: Epoch:40, Batch:839, Loss:0.300190, Acc:87.5000
Train: Epoch:40, Batch:859, Loss:0.473549, Acc:81.2500
Train: Epoch:40, Batch:879, Loss:0.654282, Acc:68.7500
Train: Epoch:40, Batch:899, Loss:0.062196, Acc:100.0000
Train_: Avg Training Loss: 0.317348, Avg Training Acc: 88.611111
Val_: Total Validation Loss: 6.634591, Acc: 18.1875
Test_: Total Testing Acc: 17.3250
Time taken = 70.6917 s

Train: Epoch:41, Batch:19, Loss:0.158557, Acc:93.7500

Train: Epoch:43, Batch:739, Loss:0.244382, Acc:93.7500
Train: Epoch:43, Batch:759, Loss:0.556784, Acc:81.2500
Train: Epoch:43, Batch:779, Loss:0.164714, Acc:100.0000
Train: Epoch:43, Batch:799, Loss:0.083952, Acc:100.0000
Train: Epoch:43, Batch:819, Loss:0.300258, Acc:81.2500
Train: Epoch:43, Batch:839, Loss:0.475685, Acc:93.7500
Train: Epoch:43, Batch:859, Loss:1.069166, Acc:81.2500
Train: Epoch:43, Batch:879, Loss:0.429205, Acc:81.2500
Train: Epoch:43, Batch:899, Loss:0.334354, Acc:81.2500
Train_: Avg Training Loss: 0.270496, Avg Training Acc: 90.361111
Val_: Total Validation Loss: 6.492287, Acc: 18.2125
Test_: Total Testing Acc: 17.4000
Time taken = 69.6990 s

Train: Epoch:44, Batch:19, Loss:0.194890, Acc:93.7500
Train: Epoch:44, Batch:39, Loss:0.090998, Acc:93.7500
Train: Epoch:44, Batch:59, Loss:0.302166, Acc:87.5000
Train: Epoch:44, Batch:79, Loss:0.540326, Acc:81.2500
Train: Epoch:44, Batch:99, Loss:0.364018, Acc:87.5000
Train: Epoch:44, Batch:119, Loss:0.181264, Acc:93.7500
Tra

Train: Epoch:46, Batch:839, Loss:0.201778, Acc:87.5000
Train: Epoch:46, Batch:859, Loss:0.500343, Acc:75.0000
Train: Epoch:46, Batch:879, Loss:0.130136, Acc:100.0000
Train: Epoch:46, Batch:899, Loss:0.132480, Acc:100.0000
Train_: Avg Training Loss: 0.253887, Avg Training Acc: 91.006944
Val_: Total Validation Loss: 6.857574, Acc: 17.7875
Test_: Total Testing Acc: 17.5125
Time taken = 68.8336 s

Train: Epoch:47, Batch:19, Loss:0.600562, Acc:68.7500
Train: Epoch:47, Batch:39, Loss:0.030443, Acc:100.0000
Train: Epoch:47, Batch:59, Loss:0.311375, Acc:81.2500
Train: Epoch:47, Batch:79, Loss:0.390516, Acc:87.5000
Train: Epoch:47, Batch:99, Loss:0.276582, Acc:81.2500
Train: Epoch:47, Batch:119, Loss:0.105356, Acc:100.0000
Train: Epoch:47, Batch:139, Loss:0.161568, Acc:93.7500
Train: Epoch:47, Batch:159, Loss:0.047293, Acc:100.0000
Train: Epoch:47, Batch:179, Loss:0.079104, Acc:93.7500
Train: Epoch:47, Batch:199, Loss:0.341101, Acc:93.7500
Train: Epoch:47, Batch:219, Loss:0.280207, Acc:81.2500


Train_: Avg Training Loss: 0.227949, Avg Training Acc: 92.319444
Val_: Total Validation Loss: 6.711306, Acc: 18.1875
Test_: Total Testing Acc: 17.4750
Time taken = 68.9121 s



In [ ]:
#Re-running the model for 40% of data
SAVE_FILE = "attm1_wren_eps50_dat40_" + time.strftime("%Y-%m-%d_%H:%M:%S", time.gmtime()) + "_" + str(args.perc_train)

for epoch in range(0, args.epochs):
    t0 = time.time()
    train(epoch, SAVE_FILE)
    avg_loss, avg_acc = validate(epoch, SAVE_FILE)
    test_acc = test(epoch, SAVE_FILE)
    model.save_model(args.save, epoch, avg_acc, avg_loss)
    print("Time taken = {:.4f} s\n".format(time.time() - t0))

Train: Epoch:0, Batch:19, Loss:2.079425, Acc:18.7500
Train: Epoch:0, Batch:39, Loss:2.050808, Acc:18.7500
Train: Epoch:0, Batch:59, Loss:2.083217, Acc:6.2500
Train: Epoch:0, Batch:79, Loss:2.125040, Acc:12.5000
Train: Epoch:0, Batch:99, Loss:2.085158, Acc:18.7500
Train: Epoch:0, Batch:119, Loss:2.044943, Acc:25.0000
Train: Epoch:0, Batch:139, Loss:2.113486, Acc:6.2500
Train: Epoch:0, Batch:159, Loss:2.091193, Acc:6.2500
Train: Epoch:0, Batch:179, Loss:2.040323, Acc:37.5000
Train: Epoch:0, Batch:199, Loss:2.062145, Acc:12.5000
Train: Epoch:0, Batch:219, Loss:2.103107, Acc:12.5000
Train: Epoch:0, Batch:239, Loss:2.054476, Acc:18.7500
Train: Epoch:0, Batch:259, Loss:2.091260, Acc:12.5000
Train: Epoch:0, Batch:279, Loss:2.089687, Acc:6.2500
Train: Epoch:0, Batch:299, Loss:2.115721, Acc:0.0000
Train: Epoch:0, Batch:319, Loss:2.057349, Acc:18.7500
Train: Epoch:0, Batch:339, Loss:2.117090, Acc:0.0000
Train: Epoch:0, Batch:359, Loss:2.089226, Acc:12.5000
Train: Epoch:0, Batch:379, Loss:2.06529

Train: Epoch:4, Batch:439, Loss:2.068060, Acc:12.5000
Train: Epoch:4, Batch:459, Loss:2.071664, Acc:6.2500
Train: Epoch:4, Batch:479, Loss:2.068025, Acc:18.7500
Train: Epoch:4, Batch:499, Loss:2.063699, Acc:25.0000
Train: Epoch:4, Batch:519, Loss:2.086209, Acc:0.0000
Train: Epoch:4, Batch:539, Loss:2.073955, Acc:12.5000
Train: Epoch:4, Batch:559, Loss:2.081711, Acc:0.0000
Train: Epoch:4, Batch:579, Loss:2.076014, Acc:12.5000
Train: Epoch:4, Batch:599, Loss:2.071354, Acc:6.2500
Train_: Avg Training Loss: 2.080468, Avg Training Acc: 12.635417
Val_: Total Validation Loss: 2.079397, Acc: 13.4375
Test_: Total Testing Acc: 12.9375
Time taken = 56.2559 s

Train: Epoch:5, Batch:19, Loss:2.078920, Acc:6.2500
Train: Epoch:5, Batch:39, Loss:2.078735, Acc:12.5000
Train: Epoch:5, Batch:59, Loss:2.072733, Acc:18.7500
Train: Epoch:5, Batch:79, Loss:2.081683, Acc:6.2500
Train: Epoch:5, Batch:99, Loss:2.081975, Acc:12.5000
Train: Epoch:5, Batch:119, Loss:2.062037, Acc:25.0000
Train: Epoch:5, Batch:139,

Train: Epoch:9, Batch:199, Loss:2.076414, Acc:6.2500
Train: Epoch:9, Batch:219, Loss:2.037529, Acc:6.2500
Train: Epoch:9, Batch:239, Loss:2.106616, Acc:12.5000
Train: Epoch:9, Batch:259, Loss:2.080511, Acc:6.2500
Train: Epoch:9, Batch:279, Loss:2.069452, Acc:6.2500
Train: Epoch:9, Batch:299, Loss:2.100311, Acc:6.2500
Train: Epoch:9, Batch:319, Loss:2.027288, Acc:31.2500
Train: Epoch:9, Batch:339, Loss:2.096567, Acc:12.5000
Train: Epoch:9, Batch:359, Loss:2.024459, Acc:12.5000
Train: Epoch:9, Batch:379, Loss:2.123945, Acc:6.2500
Train: Epoch:9, Batch:399, Loss:2.088631, Acc:0.0000
Train: Epoch:9, Batch:419, Loss:2.054700, Acc:18.7500
Train: Epoch:9, Batch:439, Loss:2.083752, Acc:18.7500
Train: Epoch:9, Batch:459, Loss:2.052301, Acc:12.5000
Train: Epoch:9, Batch:479, Loss:2.053618, Acc:25.0000
Train: Epoch:9, Batch:499, Loss:2.075387, Acc:0.0000
Train: Epoch:9, Batch:519, Loss:2.104734, Acc:12.5000
Train: Epoch:9, Batch:539, Loss:2.066010, Acc:18.7500
Train: Epoch:9, Batch:559, Loss:2.07

Train: Epoch:13, Batch:559, Loss:2.087837, Acc:25.0000
Train: Epoch:13, Batch:579, Loss:2.143429, Acc:12.5000
Train: Epoch:13, Batch:599, Loss:2.046863, Acc:18.7500
Train_: Avg Training Loss: 2.028180, Avg Training Acc: 17.541667
Val_: Total Validation Loss: 2.057979, Acc: 14.8250
Test_: Total Testing Acc: 15.8500
Time taken = 57.9281 s

Train: Epoch:14, Batch:19, Loss:2.072242, Acc:6.2500
Train: Epoch:14, Batch:39, Loss:1.982934, Acc:25.0000
Train: Epoch:14, Batch:59, Loss:2.056898, Acc:6.2500
Train: Epoch:14, Batch:79, Loss:2.018369, Acc:37.5000
Train: Epoch:14, Batch:99, Loss:2.067017, Acc:31.2500
Train: Epoch:14, Batch:119, Loss:1.902247, Acc:37.5000
Train: Epoch:14, Batch:139, Loss:2.017611, Acc:6.2500
Train: Epoch:14, Batch:159, Loss:2.008244, Acc:18.7500
Train: Epoch:14, Batch:179, Loss:1.955615, Acc:18.7500
Train: Epoch:14, Batch:199, Loss:2.035562, Acc:18.7500
Train: Epoch:14, Batch:219, Loss:2.277050, Acc:12.5000
Train: Epoch:14, Batch:239, Loss:1.990241, Acc:12.5000
Train: E

Train: Epoch:18, Batch:239, Loss:2.202578, Acc:25.0000
Train: Epoch:18, Batch:259, Loss:2.184310, Acc:12.5000
Train: Epoch:18, Batch:279, Loss:1.913831, Acc:31.2500
Train: Epoch:18, Batch:299, Loss:1.942978, Acc:31.2500
Train: Epoch:18, Batch:319, Loss:2.036726, Acc:18.7500
Train: Epoch:18, Batch:339, Loss:1.677832, Acc:50.0000
Train: Epoch:18, Batch:359, Loss:1.878170, Acc:18.7500
Train: Epoch:18, Batch:379, Loss:1.844355, Acc:12.5000
Train: Epoch:18, Batch:399, Loss:1.774600, Acc:25.0000
Train: Epoch:18, Batch:419, Loss:1.818378, Acc:37.5000
Train: Epoch:18, Batch:439, Loss:1.932743, Acc:37.5000
Train: Epoch:18, Batch:459, Loss:1.707192, Acc:25.0000
Train: Epoch:18, Batch:479, Loss:1.937702, Acc:6.2500
Train: Epoch:18, Batch:499, Loss:1.761156, Acc:50.0000
Train: Epoch:18, Batch:519, Loss:2.175096, Acc:6.2500
Train: Epoch:18, Batch:539, Loss:2.047576, Acc:31.2500
Train: Epoch:18, Batch:559, Loss:1.941324, Acc:31.2500
Train: Epoch:18, Batch:579, Loss:1.875306, Acc:25.0000
Train: Epoch

...

Train: Epoch:31, Batch:599, Loss:0.670567, Acc:75.0000
Train_: Avg Training Loss: 0.568424, Avg Training Acc: 79.208333
Val_: Total Validation Loss: 5.309543, Acc: 16.3625
Test_: Total Testing Acc: 16.3500
Time taken = 57.0667 s

Train: Epoch:32, Batch:19, Loss:0.157190, Acc:93.7500
Train: Epoch:32, Batch:39, Loss:0.127205, Acc:100.0000
Train: Epoch:32, Batch:59, Loss:1.022314, Acc:56.2500
Train: Epoch:32, Batch:79, Loss:0.428717, Acc:81.2500
Train: Epoch:32, Batch:99, Loss:0.783881, Acc:68.7500
Train: Epoch:32, Batch:119, Loss:1.068968, Acc:62.5000
Train: Epoch:32, Batch:139, Loss:0.739003, Acc:81.2500
Train: Epoch:32, Batch:159, Loss:0.161499, Acc:100.0000
Train: Epoch:32, Batch:179, Loss:0.834877, Acc:68.7500
Train: Epoch:32, Batch:199, Loss:0.425091, Acc:87.5000
Train: Epoch:32, Batch:219, Loss:0.595252, Acc:62.5000
Train: Epoch:32, Batch:239, Loss:0.270345, Acc:87.5000
Train: Epoch:32, Batch:259, Loss:0.353109, Acc:81.2500
Train: Epoch:32, Batch:279, Loss:0.544310, Acc:87.5000
Tra

Train: Epoch:36, Batch:279, Loss:0.451123, Acc:81.2500
Train: Epoch:36, Batch:299, Loss:0.600788, Acc:62.5000
Train: Epoch:36, Batch:319, Loss:0.357414, Acc:93.7500
Train: Epoch:36, Batch:339, Loss:0.476403, Acc:81.2500
Train: Epoch:36, Batch:359, Loss:0.556658, Acc:81.2500
Train: Epoch:36, Batch:379, Loss:0.362371, Acc:93.7500
Train: Epoch:36, Batch:399, Loss:0.649328, Acc:87.5000
Train: Epoch:36, Batch:419, Loss:0.321874, Acc:81.2500
Train: Epoch:36, Batch:439, Loss:0.483083, Acc:75.0000
Train: Epoch:36, Batch:459, Loss:0.699013, Acc:75.0000
Train: Epoch:36, Batch:479, Loss:0.214367, Acc:93.7500
Train: Epoch:36, Batch:499, Loss:0.647503, Acc:81.2500
Train: Epoch:36, Batch:519, Loss:0.447890, Acc:81.2500
Train: Epoch:36, Batch:539, Loss:0.348357, Acc:81.2500
Train: Epoch:36, Batch:559, Loss:0.385358, Acc:87.5000
Train: Epoch:36, Batch:579, Loss:0.385351, Acc:75.0000
Train: Epoch:36, Batch:599, Loss:0.267571, Acc:93.7500
Train_: Avg Training Loss: 0.383052, Avg Training Acc: 86.020833


Val_: Total Validation Loss: 6.490057, Acc: 16.5750
Test_: Total Testing Acc: 16.3000
Time taken = 57.5207 s

Train: Epoch:41, Batch:19, Loss:0.133839, Acc:100.0000
Train: Epoch:41, Batch:39, Loss:0.106865, Acc:100.0000
Train: Epoch:41, Batch:59, Loss:0.072106, Acc:93.7500
Train: Epoch:41, Batch:79, Loss:0.072966, Acc:100.0000
Train: Epoch:41, Batch:99, Loss:0.288332, Acc:81.2500
Train: Epoch:41, Batch:119, Loss:0.235361, Acc:93.7500
Train: Epoch:41, Batch:139, Loss:0.399269, Acc:87.5000
Train: Epoch:41, Batch:159, Loss:0.759069, Acc:93.7500
Train: Epoch:41, Batch:179, Loss:0.174113, Acc:93.7500
Train: Epoch:41, Batch:199, Loss:0.155474, Acc:93.7500
Train: Epoch:41, Batch:219, Loss:0.073731, Acc:100.0000
Train: Epoch:41, Batch:239, Loss:0.477517, Acc:87.5000
Train: Epoch:41, Batch:259, Loss:0.220623, Acc:87.5000
Train: Epoch:41, Batch:279, Loss:0.434354, Acc:68.7500
Train: Epoch:41, Batch:299, Loss:0.158404, Acc:93.7500
Train: Epoch:41, Batch:319, Loss:0.318308, Acc:93.7500
Train: Epoc

Train: Epoch:45, Batch:319, Loss:0.316771, Acc:93.7500
Train: Epoch:45, Batch:339, Loss:0.171682, Acc:93.7500
Train: Epoch:45, Batch:359, Loss:0.892003, Acc:87.5000
Train: Epoch:45, Batch:379, Loss:0.109294, Acc:100.0000
Train: Epoch:45, Batch:399, Loss:0.029284, Acc:100.0000
Train: Epoch:45, Batch:419, Loss:0.058792, Acc:100.0000
Train: Epoch:45, Batch:439, Loss:0.200256, Acc:93.7500
Train: Epoch:45, Batch:459, Loss:0.173543, Acc:87.5000
Train: Epoch:45, Batch:479, Loss:0.109164, Acc:93.7500
Train: Epoch:45, Batch:499, Loss:0.547834, Acc:87.5000
Train: Epoch:45, Batch:519, Loss:0.291152, Acc:87.5000
Train: Epoch:45, Batch:539, Loss:0.486647, Acc:75.0000
Train: Epoch:45, Batch:559, Loss:0.402010, Acc:81.2500
Train: Epoch:45, Batch:579, Loss:0.503017, Acc:87.5000
Train: Epoch:45, Batch:599, Loss:0.097187, Acc:100.0000
Train_: Avg Training Loss: 0.240289, Avg Training Acc: 92.010417
Val_: Total Validation Loss: 7.592997, Acc: 16.3625
Test_: Total Testing Acc: 16.1500
Time taken = 59.9987

Val_: Total Validation Loss: 7.622014, Acc: 16.2250
Test_: Total Testing Acc: 15.9750
Time taken = 58.9502 s



In [ ]:
#Re-running the model for 20% of data
SAVE_FILE = "attm1_wren_eps50_dat20_" + time.strftime("%Y-%m-%d_%H:%M:%S", time.gmtime()) + "_" + str(args.perc_train)

for epoch in range(0, args.epochs):
    t0 = time.time()
    train(epoch, SAVE_FILE)
    avg_loss, avg_acc = validate(epoch, SAVE_FILE)
    test_acc = test(epoch, SAVE_FILE)
    model.save_model(args.save, epoch, avg_acc, avg_loss)
    print("Time taken = {:.4f} s\n".format(time.time() - t0))

Train: Epoch:0, Batch:19, Loss:2.109465, Acc:12.5000
Train: Epoch:0, Batch:39, Loss:2.028800, Acc:25.0000
Train: Epoch:0, Batch:59, Loss:2.075105, Acc:12.5000
Train: Epoch:0, Batch:79, Loss:2.096509, Acc:12.5000
Train: Epoch:0, Batch:99, Loss:2.091674, Acc:12.5000
Train: Epoch:0, Batch:119, Loss:2.070059, Acc:25.0000
Train: Epoch:0, Batch:139, Loss:2.099438, Acc:0.0000
Train: Epoch:0, Batch:159, Loss:2.088194, Acc:0.0000
Train: Epoch:0, Batch:179, Loss:2.071148, Acc:18.7500
Train: Epoch:0, Batch:199, Loss:2.071145, Acc:6.2500
Train: Epoch:0, Batch:219, Loss:2.027119, Acc:31.2500
Train: Epoch:0, Batch:239, Loss:2.063850, Acc:0.0000
Train: Epoch:0, Batch:259, Loss:2.089536, Acc:12.5000
Train: Epoch:0, Batch:279, Loss:2.052868, Acc:25.0000
Train: Epoch:0, Batch:299, Loss:2.127280, Acc:0.0000
Train_: Avg Training Loss: 2.086318, Avg Training Acc: 12.000000
Val_: Total Validation Loss: 2.079398, Acc: 12.6625
Test_: Total Testing Acc: 13.3875
Time taken = 48.8979 s

Train: Epoch:1, Batch:19,

Train: Epoch:8, Batch:179, Loss:2.077973, Acc:12.5000
Train: Epoch:8, Batch:199, Loss:2.087721, Acc:0.0000
Train: Epoch:8, Batch:219, Loss:2.071918, Acc:18.7500
Train: Epoch:8, Batch:239, Loss:2.082228, Acc:12.5000
Train: Epoch:8, Batch:259, Loss:2.074320, Acc:25.0000
Train: Epoch:8, Batch:279, Loss:2.076939, Acc:12.5000
Train: Epoch:8, Batch:299, Loss:2.082292, Acc:6.2500
Train_: Avg Training Loss: 2.080597, Avg Training Acc: 12.333333
Val_: Total Validation Loss: 2.079155, Acc: 12.8250
Test_: Total Testing Acc: 13.2000
Time taken = 50.5619 s

Train: Epoch:9, Batch:19, Loss:2.072995, Acc:18.7500
Train: Epoch:9, Batch:39, Loss:2.082222, Acc:6.2500
Train: Epoch:9, Batch:59, Loss:2.094512, Acc:6.2500
Train: Epoch:9, Batch:79, Loss:2.075098, Acc:18.7500
Train: Epoch:9, Batch:99, Loss:2.098991, Acc:6.2500
Train: Epoch:9, Batch:119, Loss:2.079552, Acc:18.7500
Train: Epoch:9, Batch:139, Loss:2.075360, Acc:6.2500
Train: Epoch:9, Batch:159, Loss:2.078586, Acc:12.5000
Train: Epoch:9, Batch:179,

Train: Epoch:16, Batch:299, Loss:1.923432, Acc:6.2500
Train_: Avg Training Loss: 1.980144, Avg Training Acc: 20.375000
Val_: Total Validation Loss: 2.093844, Acc: 14.0625
Test_: Total Testing Acc: 14.6625
Time taken = 49.1047 s

Train: Epoch:17, Batch:19, Loss:1.812599, Acc:43.7500
Train: Epoch:17, Batch:39, Loss:1.810312, Acc:31.2500
Train: Epoch:17, Batch:59, Loss:1.649593, Acc:18.7500
Train: Epoch:17, Batch:79, Loss:1.887121, Acc:31.2500
Train: Epoch:17, Batch:99, Loss:2.006530, Acc:12.5000
Train: Epoch:17, Batch:119, Loss:2.115898, Acc:0.0000
Train: Epoch:17, Batch:139, Loss:2.095706, Acc:12.5000
Train: Epoch:17, Batch:159, Loss:1.848756, Acc:18.7500
Train: Epoch:17, Batch:179, Loss:2.015419, Acc:6.2500
Train: Epoch:17, Batch:199, Loss:2.007317, Acc:0.0000
Train: Epoch:17, Batch:219, Loss:1.926150, Acc:37.5000
Train: Epoch:17, Batch:239, Loss:1.921148, Acc:31.2500
Train: Epoch:17, Batch:259, Loss:1.936716, Acc:25.0000
Train: Epoch:17, Batch:279, Loss:1.751826, Acc:18.7500
Train: Ep

Train: Epoch:25, Batch:39, Loss:1.058349, Acc:62.5000
Train: Epoch:25, Batch:59, Loss:0.658914, Acc:75.0000
Train: Epoch:25, Batch:79, Loss:0.719589, Acc:81.2500
Train: Epoch:25, Batch:99, Loss:0.870014, Acc:62.5000
Train: Epoch:25, Batch:119, Loss:0.882000, Acc:68.7500
Train: Epoch:25, Batch:139, Loss:1.058017, Acc:62.5000
Train: Epoch:25, Batch:159, Loss:0.959494, Acc:62.5000
Train: Epoch:25, Batch:179, Loss:0.915428, Acc:56.2500
Train: Epoch:25, Batch:199, Loss:0.889977, Acc:56.2500
Train: Epoch:25, Batch:219, Loss:0.524285, Acc:87.5000
Train: Epoch:25, Batch:239, Loss:1.068804, Acc:68.7500
Train: Epoch:25, Batch:259, Loss:0.967775, Acc:75.0000
Train: Epoch:25, Batch:279, Loss:0.669436, Acc:81.2500
Train: Epoch:25, Batch:299, Loss:0.795796, Acc:75.0000
Train_: Avg Training Loss: 0.875838, Avg Training Acc: 67.958333
Val_: Total Validation Loss: 3.979101, Acc: 15.4500
Test_: Total Testing Acc: 15.1250
Time taken = 47.5768 s

Train: Epoch:26, Batch:19, Loss:0.804157, Acc:75.0000
Train

Train: Epoch:33, Batch:139, Loss:0.153446, Acc:93.7500
Train: Epoch:33, Batch:159, Loss:0.125506, Acc:93.7500
Train: Epoch:33, Batch:179, Loss:0.300638, Acc:87.5000
Train: Epoch:33, Batch:199, Loss:0.426557, Acc:81.2500
Train: Epoch:33, Batch:219, Loss:0.308134, Acc:81.2500
Train: Epoch:33, Batch:239, Loss:0.289601, Acc:87.5000
Train: Epoch:33, Batch:259, Loss:0.127559, Acc:93.7500
Train: Epoch:33, Batch:279, Loss:0.893597, Acc:62.5000
Train: Epoch:33, Batch:299, Loss:0.216901, Acc:87.5000
Train_: Avg Training Loss: 0.316918, Avg Training Acc: 89.187500
Val_: Total Validation Loss: 8.191050, Acc: 14.8375
Test_: Total Testing Acc: 15.1125
Time taken = 46.8835 s

Train: Epoch:34, Batch:19, Loss:0.304349, Acc:100.0000
Train: Epoch:34, Batch:39, Loss:0.142196, Acc:93.7500
Train: Epoch:34, Batch:59, Loss:0.176646, Acc:93.7500
Train: Epoch:34, Batch:79, Loss:0.215246, Acc:87.5000
Train: Epoch:34, Batch:99, Loss:0.255295, Acc:93.7500
Train: Epoch:34, Batch:119, Loss:0.135649, Acc:93.7500
Trai

Train: Epoch:41, Batch:219, Loss:0.171670, Acc:87.5000
Train: Epoch:41, Batch:239, Loss:0.040782, Acc:100.0000
Train: Epoch:41, Batch:259, Loss:0.101475, Acc:100.0000
Train: Epoch:41, Batch:279, Loss:0.066034, Acc:100.0000
Train: Epoch:41, Batch:299, Loss:0.143478, Acc:93.7500
Train_: Avg Training Loss: 0.161804, Avg Training Acc: 94.750000
Val_: Total Validation Loss: 9.425359, Acc: 15.2125
Test_: Total Testing Acc: 15.3750
Time taken = 48.0707 s

Train: Epoch:42, Batch:19, Loss:0.133214, Acc:93.7500
Train: Epoch:42, Batch:39, Loss:0.037814, Acc:100.0000
Train: Epoch:42, Batch:59, Loss:0.059166, Acc:100.0000
Train: Epoch:42, Batch:79, Loss:0.263123, Acc:93.7500
Train: Epoch:42, Batch:99, Loss:0.104611, Acc:93.7500
Train: Epoch:42, Batch:119, Loss:0.162600, Acc:87.5000
Train: Epoch:42, Batch:139, Loss:0.150834, Acc:93.7500
Train: Epoch:42, Batch:159, Loss:0.069010, Acc:100.0000
Train: Epoch:42, Batch:179, Loss:0.170767, Acc:87.5000
Train: Epoch:42, Batch:199, Loss:0.236469, Acc:93.7500

Train: Epoch:49, Batch:299, Loss:0.030727, Acc:100.0000
Train_: Avg Training Loss: 0.127437, Avg Training Acc: 96.187500
Val_: Total Validation Loss: 11.367435, Acc: 15.0000
Test_: Total Testing Acc: 14.3875
Time taken = 46.9891 s



#### Test on Center Single, 2x2 Grid, Left-Right, Out-In

In [ ]:
testset = dataset(args.path, "test", args.img_size, transform=transforms.Compose([ToTensor()]))

In [ ]:
center_single_fnames = [x for x in testset.file_names if '/center_single/' in x]
distribute_four_fnames = [x for x in testset.file_names if '/distribute_four/' in x]
in_distribute_four_out_center_single_fnames = \
    [x for x in testset.file_names if '/in_distribute_four_out_center_single/' in x]
left_center_single_right_center_single_fnames = \
    [x for x in testset.file_names if '/left_center_single_right_center_single/' in x]

In [ ]:
print(len(center_single_fnames), \
len(distribute_four_fnames), \
len(in_distribute_four_out_center_single_fnames), \
len(left_center_single_right_center_single_fnames))

2000 2000 2000 2000


In [ ]:
cs_set = dataset(args.path, "test", args.img_size, transform=transforms.Compose([ToTensor()]))
cs_set.file_names = center_single_fnames
cs_testloader = DataLoader(cs_set, batch_size=args.batch_size, shuffle=False, num_workers=80)

df_set = dataset(args.path, "test", args.img_size, transform=transforms.Compose([ToTensor()]))
df_set.file_names = distribute_four_fnames
df_testloader = DataLoader(df_set, batch_size=args.batch_size, shuffle=False, num_workers=80)

idfo_set = dataset(args.path, "test", args.img_size, transform=transforms.Compose([ToTensor()]))
idfo_set.file_names = in_distribute_four_out_center_single_fnames
idfo_testloader = DataLoader(idfo_set, batch_size=args.batch_size, shuffle=False, num_workers=80)

lcsr_set = dataset(args.path, "test", args.img_size, transform=transforms.Compose([ToTensor()]))
lcsr_set.file_names = left_center_single_right_center_single_fnames
lcsr_testloader = DataLoader(lcsr_set, batch_size=args.batch_size, shuffle=False, num_workers=80)

In [ ]:
def test_sep(epoch, tl, save_file):
    model.eval()
    accuracy = 0

    acc_all = 0.0
    counter = 0
    for batch_idx, (image, target, meta_target) in enumerate(tl):
        counter += 1
        if args.cuda:
            image = image.cuda()
            target = target.cuda()
            meta_target = meta_target.cuda()
            #meta_structure = meta_structure.cuda()
            #embedding = embedding.cuda()
            #indicator = indicator.cuda()
        acc = model.test_(image, target, meta_target)#, meta_structure, embedding, indicator)
        # print('Test: Epoch:{}, Batch:{}, Acc:{:.4f}.'.format(epoch, batch_idx, acc))  
        acc_all += acc
    if counter > 0:
        save_str = "Test_: Total Testing Acc: {:.4f}".format(acc_all / float(counter))
        print(save_str)
        with open(save_file, 'a') as f:
            f.write(save_str + "\n")
    return acc_all/float(counter)

In [ ]:
SAVE_FILE_SEP = SAVE_FILE + "_sep"

test_acc_cs = test_sep(epoch,cs_testloader, SAVE_FILE_SEP)
test_acc_df = test_sep(epoch, df_testloader, SAVE_FILE_SEP)
test_acc_idfo = test_sep(epoch, idfo_testloader, SAVE_FILE_SEP)
test_acc_lcsr = test_sep(epoch, lcsr_testloader, SAVE_FILE_SEP)

Test_: Total Testing Acc: 23.6500
Test_: Total Testing Acc: 21.5000
Test_: Total Testing Acc: 17.2500
Test_: Total Testing Acc: 18.8000


In [ ]:
for batch_idx, (image, target, meta_target, meta_structure, embedding, indicator) in enumerate(cs_testloader):
    print("Meta Target:", meta_target)